In [22]:
from pyspark.sql import SparkSession
import numpy as np

In [23]:
spark = SparkSession.builder.appName('COVID19').config("spark.executor.memory", "4g").config("spark.driver.memory", "4g").getOrCreate()

ruta = '../../Data/DatosCovid/220720COVID19MEXICO.csv'
df = spark.read.csv(
    ruta,
    header = True,
    inferSchema = False,    # evita que Spark adivine tipos (así no se confunde)
    mode = "DROPMALFORMED", # elimina filas corruptas desde el inicio
    multiLine = False,      # evita que las comas o saltos de línea rompan filas
    escape="\"",          # ignora comillas mal cerradas
)
df.show()

ERROR:root:KeyboardInterrupt while sending command.                             
Traceback (most recent call last):
  File "/home/jair/anaconda3/envs/BigData/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jair/anaconda3/envs/BigData/lib/python3.11/site-packages/py4j/clientserver.py", line 535, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jair/anaconda3/envs/BigData/lib/python3.11/socket.py", line 718, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
type(df)

pyspark.sql.classic.dataframe.DataFrame

In [ ]:
print(f'Registros antes de eliminar algunos: {df.count()}')

Registros antes de eliminar algunos: 17311042


### Eliminar columnas que no han de servir

In [ ]:
fechas = [col for col in df.columns if 'FECHA_' in col]
eliminar = ['ID_REGISTRO'] + fechas
df = df.drop(*eliminar)
df.show()


+------+------+----------+----+-----------+-----------+-------------+-------------+--------+--------+----+------------+--------+------------------+--------+--------+----+----+--------+------------+--------+--------------+--------+-------------+----------+---------+----------------+-------------+---------------------+------------------+-------------------+--------+-----------------+-----------+---+
|ORIGEN|SECTOR|ENTIDAD_UM|SEXO|ENTIDAD_NAC|ENTIDAD_RES|MUNICIPIO_RES|TIPO_PACIENTE|INTUBADO|NEUMONIA|EDAD|NACIONALIDAD|EMBARAZO|HABLA_LENGUA_INDIG|INDIGENA|DIABETES|EPOC|ASMA|INMUSUPR|HIPERTENSION|OTRA_COM|CARDIOVASCULAR|OBESIDAD|RENAL_CRONICA|TABAQUISMO|OTRO_CASO|TOMA_MUESTRA_LAB|RESULTADO_LAB|TOMA_MUESTRA_ANTIGENO|RESULTADO_ANTIGENO|CLASIFICACION_FINAL|MIGRANTE|PAIS_NACIONALIDAD|PAIS_ORIGEN|UCI|
+------+------+----------+----+-----------+-----------+-------------+-------------+--------+--------+----+------------+--------+------------------+--------+--------+----+----+--------+------------+-

In [ ]:
df.dtypes

[('ORIGEN', 'string'),
 ('SECTOR', 'string'),
 ('ENTIDAD_UM', 'string'),
 ('SEXO', 'string'),
 ('ENTIDAD_NAC', 'string'),
 ('ENTIDAD_RES', 'string'),
 ('MUNICIPIO_RES', 'string'),
 ('TIPO_PACIENTE', 'string'),
 ('INTUBADO', 'string'),
 ('NEUMONIA', 'string'),
 ('EDAD', 'string'),
 ('NACIONALIDAD', 'string'),
 ('EMBARAZO', 'string'),
 ('HABLA_LENGUA_INDIG', 'string'),
 ('INDIGENA', 'string'),
 ('DIABETES', 'string'),
 ('EPOC', 'string'),
 ('ASMA', 'string'),
 ('INMUSUPR', 'string'),
 ('HIPERTENSION', 'string'),
 ('OTRA_COM', 'string'),
 ('CARDIOVASCULAR', 'string'),
 ('OBESIDAD', 'string'),
 ('RENAL_CRONICA', 'string'),
 ('TABAQUISMO', 'string'),
 ('OTRO_CASO', 'string'),
 ('TOMA_MUESTRA_LAB', 'string'),
 ('RESULTADO_LAB', 'string'),
 ('TOMA_MUESTRA_ANTIGENO', 'string'),
 ('RESULTADO_ANTIGENO', 'string'),
 ('CLASIFICACION_FINAL', 'string'),
 ('MIGRANTE', 'string'),
 ('PAIS_NACIONALIDAD', 'string'),
 ('PAIS_ORIGEN', 'string'),
 ('UCI', 'string')]

### Cambiar el formato de algunas columnas que deberían ser numericas

In [ ]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col

numericas = [col for col in df.columns if col != 'PAIS_NACIONALIDAD']
for c in numericas:
    df = df.withColumn(c, col(c).cast(IntegerType()))
    
df.dtypes

[('ORIGEN', 'int'),
 ('SECTOR', 'int'),
 ('ENTIDAD_UM', 'int'),
 ('SEXO', 'int'),
 ('ENTIDAD_NAC', 'int'),
 ('ENTIDAD_RES', 'int'),
 ('MUNICIPIO_RES', 'int'),
 ('TIPO_PACIENTE', 'int'),
 ('INTUBADO', 'int'),
 ('NEUMONIA', 'int'),
 ('EDAD', 'int'),
 ('NACIONALIDAD', 'int'),
 ('EMBARAZO', 'int'),
 ('HABLA_LENGUA_INDIG', 'int'),
 ('INDIGENA', 'int'),
 ('DIABETES', 'int'),
 ('EPOC', 'int'),
 ('ASMA', 'int'),
 ('INMUSUPR', 'int'),
 ('HIPERTENSION', 'int'),
 ('OTRA_COM', 'int'),
 ('CARDIOVASCULAR', 'int'),
 ('OBESIDAD', 'int'),
 ('RENAL_CRONICA', 'int'),
 ('TABAQUISMO', 'int'),
 ('OTRO_CASO', 'int'),
 ('TOMA_MUESTRA_LAB', 'int'),
 ('RESULTADO_LAB', 'int'),
 ('TOMA_MUESTRA_ANTIGENO', 'int'),
 ('RESULTADO_ANTIGENO', 'int'),
 ('CLASIFICACION_FINAL', 'int'),
 ('MIGRANTE', 'int'),
 ('PAIS_NACIONALIDAD', 'string'),
 ('PAIS_ORIGEN', 'int'),
 ('UCI', 'int')]

In [ ]:
df.groupBy('PAIS_NACIONALIDAD').count().orderBy('count', ascending = False).show()

+--------------------+--------+
|   PAIS_NACIONALIDAD|   count|
+--------------------+--------+
|              México|17186453|
|Estados Unidos de...|   51950|
|           Venezuela|    9242|
|            Colombia|    7410|
|República de Hond...|    5059|
|           Guatemala|    4412|
|                Cuba|    3193|
|           Argentina|    2865|
|           Nicaragua|    2835|
|              España|    2825|
|         El Salvador|    2750|
|              Brasil|    2703|
|            Alemania|    2165|
|                Otro|    2104|
|             Francia|    1903|
|               Haití|    1847|
|              Canadá|    1795|
|                Perú|    1793|
|               China|    1782|
|             Ecuador|    1768|
+--------------------+--------+
only showing top 20 rows


### COdificar la columna de pais nacionalidad

In [24]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol='PAIS_NACIONALIDAD',
                       outputCol='PAIS_NACIONALIDAD_IDX',
                       handleInvalid="skip" )

df = indexer.fit(df).transform(df)

df.select('PAIS_NACIONALIDAD', 'PAIS_NACIONALIDAD_IDX').show()

+-----------------+---------------------+
|PAIS_NACIONALIDAD|PAIS_NACIONALIDAD_IDX|
+-----------------+---------------------+
|           México|                  0.0|
|           México|                  0.0|
|           México|                  0.0|
|           México|                  0.0|
|           México|                  0.0|
|           México|                  0.0|
|           México|                  0.0|
|           México|                  0.0|
|           México|                  0.0|
|           México|                  0.0|
|           México|                  0.0|
|           México|                  0.0|
|           México|                  0.0|
|           México|                  0.0|
|           México|                  0.0|
|           México|                  0.0|
|           México|                  0.0|
|           México|                  0.0|
|           México|                  0.0|
|           México|                  0.0|
+-----------------+---------------

In [25]:
df = df.drop('PAIS_NACIONALIDAD')
df.show()

+-------------------+-----------+------+------+----------+----+-----------+-----------+-------------+-------------+-------------+--------------+----------+--------+--------+----+------------+--------+------------------+--------+--------+----+----+--------+------------+--------+--------------+--------+-------------+----------+---------+----------------+-------------+---------------------+------------------+-------------------+--------+-----------+---+---------------------+
|FECHA_ACTUALIZACION|ID_REGISTRO|ORIGEN|SECTOR|ENTIDAD_UM|SEXO|ENTIDAD_NAC|ENTIDAD_RES|MUNICIPIO_RES|TIPO_PACIENTE|FECHA_INGRESO|FECHA_SINTOMAS| FECHA_DEF|INTUBADO|NEUMONIA|EDAD|NACIONALIDAD|EMBARAZO|HABLA_LENGUA_INDIG|INDIGENA|DIABETES|EPOC|ASMA|INMUSUPR|HIPERTENSION|OTRA_COM|CARDIOVASCULAR|OBESIDAD|RENAL_CRONICA|TABAQUISMO|OTRO_CASO|TOMA_MUESTRA_LAB|RESULTADO_LAB|TOMA_MUESTRA_ANTIGENO|RESULTADO_ANTIGENO|CLASIFICACION_FINAL|MIGRANTE|PAIS_ORIGEN|UCI|PAIS_NACIONALIDAD_IDX|
+-------------------+-----------+------+------

In [26]:
df = df.withColumn('PAIS_NACIONALIDAD_IDX', col('PAIS_NACIONALIDAD_IDX').cast(IntegerType()))
df.dtypes

[('FECHA_ACTUALIZACION', 'string'),
 ('ID_REGISTRO', 'string'),
 ('ORIGEN', 'string'),
 ('SECTOR', 'string'),
 ('ENTIDAD_UM', 'string'),
 ('SEXO', 'string'),
 ('ENTIDAD_NAC', 'string'),
 ('ENTIDAD_RES', 'string'),
 ('MUNICIPIO_RES', 'string'),
 ('TIPO_PACIENTE', 'string'),
 ('FECHA_INGRESO', 'string'),
 ('FECHA_SINTOMAS', 'string'),
 ('FECHA_DEF', 'string'),
 ('INTUBADO', 'string'),
 ('NEUMONIA', 'string'),
 ('EDAD', 'string'),
 ('NACIONALIDAD', 'string'),
 ('EMBARAZO', 'string'),
 ('HABLA_LENGUA_INDIG', 'string'),
 ('INDIGENA', 'string'),
 ('DIABETES', 'string'),
 ('EPOC', 'string'),
 ('ASMA', 'string'),
 ('INMUSUPR', 'string'),
 ('HIPERTENSION', 'string'),
 ('OTRA_COM', 'string'),
 ('CARDIOVASCULAR', 'string'),
 ('OBESIDAD', 'string'),
 ('RENAL_CRONICA', 'string'),
 ('TABAQUISMO', 'string'),
 ('OTRO_CASO', 'string'),
 ('TOMA_MUESTRA_LAB', 'string'),
 ('RESULTADO_LAB', 'string'),
 ('TOMA_MUESTRA_ANTIGENO', 'string'),
 ('RESULTADO_ANTIGENO', 'string'),
 ('CLASIFICACION_FINAL', 'string'

In [27]:
df.groupBy('PAIS_NACIONALIDAD_IDX').count().show()

+---------------------+-----+
|PAIS_NACIONALIDAD_IDX|count|
+---------------------+-----+
|                   31|  251|
|                  137|    5|
|                   65|   53|
|                   53|   75|
|                   78|   34|
|                  155|    2|
|                   34|  232|
|                   81|   31|
|                   28|  423|
|                  183|    1|
|                   26|  488|
|                   27|  451|
|                  159|    2|
|                   44|  134|
|                   12| 2165|
|                   91|   23|
|                   22| 1235|
|                  122|    6|
|                   93|   22|
|                  111|   11|
+---------------------+-----+
only showing top 20 rows
